In [9]:
from dotenv import load_dotenv
import warnings

from environment.game import Game
from environment.game_state import GameState
from environment.generators.random_generator import RandomCrimeSceneMapGenerator
from llm.llama import Llama
from llm.llm_story_generator import LlmStoryGenerator

load_dotenv(dotenv_path="./llm-dungeon-adventures/.env")
warnings.filterwarnings("ignore", category=FutureWarning)

ModuleNotFoundError: No module named 'environment.generators.random_generator'

1. Create a room layout and a theme

In [8]:
number_of_rooms = 3
theme = "Library of Alexandria, 340 BC, crazy librarian"

crime_scene_map = RandomGridGenerator(seed=42).generate(number_of_rooms)

2. Init Llama2

In [3]:
llm = Llama()

Loading checkpoint shards: 100%|██████████| 3/3 [00:31<00:00, 10.61s/it]


Model loaded on cuda:0


3. Create a story

In [4]:
story = LlmStoryGenerator(rooms_layout, llm).create_new_story(number_of_suspects=number_of_rooms - 1, theme=theme, dummy=False)

story

{'theme': 'Library of Alexandria, 340 BC, crazy librarian',
 'killer': {'name': 'Gaius',
  'age': 40,
  'occupation': 'Crazy Librarian',
  'motive': "Gaius became increasingly obsessed with ancient and forbidden knowledge. He believed that by eliminating anyone who questioned him, he could protect the library's secrets.\n",
  'alibi': 'Gaius claims he was in his secret chamber, delving into forbidden texts. No one can vouch for his whereabouts.\n'},
 'victim': {'name': 'Archibald Ptolemy',
  'age': 47,
  'occupation': 'Head Librarian',
  'murder_weapon': 'Ancient scroll with poisoned ink',
  'death_description': 'Found dead in his office surrounded by stacks of books, face contorted in a mixture of shock and horror as if he had been reading one too many forbidden tomes.'},
 'suspects': [{'name': 'Eutropia',
   'age': 28,
   'occupation': 'Junior Librarian',
   'motive': 'Eutropia harbored resentment towards Archibald due to his frequent criticism of her cataloging methods and habitual 

4. Create an initial game state, then create the game

In [5]:
initial_game_state = GameState(crime_scene_map, story)
game = Game(initial_game_state)

NameError: name 'generator' is not defined